In [ ]:
# for mooring, can we just interpolate the gridded fields directly rather than interpolating to points and then re-gridding?
# YES but it's slower (!?!)
X = ds.XC.isel(time=0) 
Y = ds.YC.isel(time=0)
i = ds.i
j = ds.j
z = ds.Z.isel(time=0)
k = ds.k
f_x = interpolate.interp1d(X[0,:].values, i)
f_y = interpolate.interp1d(Y[:,0].values, j)
f_z = interpolate.interp1d(z, k, bounds_error=False)

#
model_xav = ds.XC.isel(time=0, j=0).mean(dim='i').values
model_yav = ds.YC.isel(time=0, i=0).mean(dim='j').values
xmooring = model_xav # default lat/lon is the center of the domain
ymooring = model_yav
zmooring_TS = [-1, -10, -50, -100] # depth of T/S instruments
zmooring_UV = [-1, -10, -50, -100] # depth of U/V instruments

#
# ds.Theta.sel(k=1)
# ds.Theta.sel(k=1)
ki = f_z(zmooring_TS[0])
ii = f_x(xmooring)
ji = f_y(ymooring)
xo = ds.interp(i=ii, j=ji, k=ki)
xo.Theta.plot.line(x='time')
sdata.Theta.where(sdata.dep == sdata.dep.max(), drop=True).plot.line(x='time')
sgridded.Theta.isel(depth=0).plot()

In [ ]:
# test if the above regridding is correct by plotting the data at one level as a time series
# YES.

# shallowest (max because negative)
zpl = np.max(sdata['dep'].data)
print(zpl)

# select data at one depth:
this_var_onedep = sdata[vbl].where(sdata.dep == zpl, drop=True)

# fixed data at the same depth
this_var_fixed_onedep = this_var_fix[0,:]


# # deepest
# zpl = np.min(sdata['dep'].data)
# this_var_onedep = sdata[vbl].where(sdata.dep == zpl, drop=True)
# this_var_reshaped_onedep = this_var_reshape[494,:]
# print(zpl)


tpl = np.linspace(0,np.max(sdata['time']),len(this_var_reshaped_onedep))


plt.plot(sdata['time'].where(sdata.dep == zpl, drop=True).data, this_var_onedep.data,'.')
plt.plot(tpl,this_var_fixed_onedep,'r-.')

plt.show()

In [16]:
# --------------------------------------------------------------------
# USER INPUTS:
# --------------------------------------------------------------------
import datetime
# specify region from this list:
# WesternMed  ROAM_MIZ  NewCaledonia  NWPacific  BassStrait  RockallTrough  ACC_SMST
# MarmaraSea  LabradorSea  CapeBasin
RegionName = 'ACC_SMST' 

# specify date range as start date & number of days.
start_date = datetime.date(2012,1,1)
ndays = 31
# ndays = 2
# ndays = 10

# directory where data files are stored
# datadir = './data/' # default is ./data
datadir = '/data1/adac/mitgcm/netcdf/' + RegionName + '/'
outputdir = '/data1/adac/osse_output/' + RegionName + '/'
argodir = '/data1/argo/gridded/' # store argo climatology here

# optional details for sampling (if not specified, reasonable defaults will be used)
sampling_details = {
    'SAMPLING_STRATEGY' : 'sim_uctd', # options: sim_glider, sim_uctd or trajectory_file.add: mooring, ASV
#     'SAMPLING_STRATEGY' : 'sim_mooring', # options: sim_glider, sim_uctd or trajectory_file.add: mooring, ASV
    'PATTERN' : 'lawnmower', # back-forth or lawnmower 
    'zrange' : [-1, -1000],  # depth range of T/S profiles (down is negative). * add U/V range? *
    #'hspeed' : 0.25,  # platform horizontal speed in m/s
    #'vspeed' : 0.1, # platform vertical (profile) speed in m/s 
    #'trajectory_file' : None, # if SAMPLING_STRATEGY = 'trajectory_file', specify trajectory file
    #'AT_END' : 'repeat' # behaviour at and of trajectory: 'reverse', 'repeat' or 'terminate'. (could also 'restart'?)
}




sampling_details['SAMPLING_STRATEGY']
end_date = start_date + datetime.timedelta(days=ndays)
 
filename_out = (f'OSSE_{RegionName}_{sampling_details["SAMPLING_STRATEGY"]}_{start_date}_to_{end_date}')
# filename_out = (f'OSSE_{RegionName}_{sampling_details["SAMPLING_STRATEGY"]}')

print(filename_out)

OSSE_ACC_SMST_sim_uctd_2012-01-01_to_2012-02-01


#### EXPLORE DIFFERENT INTERPOLATION METHODS 

It's re-gridding the 3-d interpolated data that is slow. (2d is trivial)

Which part?
the line this_var = subsampled_data[vbl].data.compute().copy() 
is SUPER slow (20 sec) for V, slow (9 sec) for U, moderate (2 sec) for theta and salt, 0 for SH and vorticity

I guess because making those regridding computations is slow?
- is the regridding necessary? (yes, to do it right - but could check)
- can we regrid when we derive fields? (yes - but not sure it's any faster)


In [ ]:
# ORIGINAL CODE in osse_tools.py
start_time = time.time()
# vbls3d = ['U','V', 'Theta','Salt','vorticity','steric_height']
# vbls2d = ['steric_height_true', 'Eta', 'KPPhbl', 'PhiBot', 'oceFWflx', 'oceQnet', 'oceQsw', 'oceSflux']


vbls3d = ['U_c','V_c', 'U', 'V']
vbls3d = ['U', 'V']

# -----

# ------Regrid the data to depth/time (3-d fields) or subsample to time (2-d fields)
print('Gridding the interpolated data...')
# get times associated with profiles:
SAMPLING_STRATEGY = survey_track['SAMPLING_STRATEGY']
if SAMPLING_STRATEGY == 'sim_mooring':
    # - for mooring, use the subsampled time grid:
    times = np.unique(subsampled_data.time.values)
else:
    # -- for glider/uctd, take the shallowest & deepest profiles (every second value, since top/bottom get sampled twice for each profile)
    time_deepest = subsampled_data.time.where(subsampled_data.dep == subsampled_data.dep.min(), drop=True).values[0:-1:2]
    time_shallowest = subsampled_data.time.where(subsampled_data.dep == subsampled_data.dep.max(), drop=True).values[0:-1:2]
    times = np.sort(np.concatenate((time_shallowest, time_deepest)))
    # this results in a time grid that may not be uniformly spaced, but is correct
    # - for a uniform grid, use the mean time spacing - may not be perfectly accurate, but is evenly spaced
    dt = np.mean(np.diff(time_shallowest))/2 # average spacing of profiles (half of one up/down, so divide by two)
    times_uniform = np.arange(survey_track.n_profiles.values*2) * dt
    print("--- deal with times: %s seconds ---" % int(time.time() - start_time))
# nt is the number of profiles (times):
nt = len(times)  
# xgr is the vertical grid; nz is the number of depths for each profile
# depths are negative, so sort in reverse order using flip
zgridded = np.flip(np.unique(subsampled_data.dep.data))
nz = int(len(zgridded))

# -- initialize the dataset:
sgridded = xr.Dataset(
    coords = dict(depth=(["depth"],zgridded),
              time=(["time"],times))
)
# -- 3-d fields: loop & reshape 3-d data from profiles to a 2-d (depth-time) grid:
# first, extract each variable, then reshape to a grid
# add U and V to the list:
# vbls3d.append('U')
# vbls3d.append('V')


for vbl in vbls3d:
    print(vbl)
    start_time = time.time()
    this_var = subsampled_data[vbl].data.compute().copy() 
    print("--- this_var = subsampled_data[vbl].data.compute().copy(): %s seconds ---" % int(time.time() - start_time))

    # reshape to nz,nt
    start_time = time.time()
    this_var_reshape = np.reshape(this_var,(nz,nt), order='F') # fortran order is important!
    print("--- this_var_reshape = np.reshape(this_var,(nz,nt), order='F'): %s seconds ---" % int(time.time() - start_time))
    # for platforms with up & down profiles (uCTD and glider),
    # every second column is upside-down (upcast data)
    # starting with the first column, flip the data upside down so that upcasts go from top to bottom
    start_time = time.time()
    if SAMPLING_STRATEGY != 'sim_mooring':
        this_var_fix = this_var_reshape.copy()
        #this_var_fix[:,0::2] = this_var_fix[-1::-1,0::2] 
        this_var_fix[:,1::2] = this_var_fix[-1::-1,1::2]  # Starting with SECOND column
        sgridded[vbl] = (("depth","time"), this_var_fix)
    elif SAMPLING_STRATEGY == 'sim_mooring':
        sgridded[vbl] = (("depth","time"), this_var_reshape)
        
    print("--- this_var_fix[:,1::2] = this_var_fix[-1::-1,1::2]: %s seconds ---" % int(time.time() - start_time))
print("--- 3d vars: TOTAL %s seconds ---" % int(time.time() - start_time))

# # for sampled steric height, we want the value integrated from the deepest sampling depth:
# sgridded['steric_height'] = (("time"), sgridded['steric_height'].isel(depth=nz-1))
# # rename to "sampled" for clarity
# sgridded.rename_vars({'steric_height':'steric_height_sampled'})


In [ ]:
subsampled_data

In [ ]:
## Create a new dataset to contain the interpolated data, and interpolate
subsampled_data = xr.Dataset(
    dict(
        t = xr.DataArray(survey_track.time, dims='points'), # call this time, for now, so that the interpolation works
        lon = xr.DataArray(survey_track.lon, dims='points'),
        lat = xr.DataArray(survey_track.lat, dims='points'),
        dep = xr.DataArray(survey_track.dep, dims='points'),
        points = xr.DataArray(survey_track.points, dims='points')
    )
)

print('Interpolating model fields to the sampling track...')
# loop & interpolate through 3d variables:
vbls3d = ['Theta','Salt','vorticity','steric_height']
for vbl in vbls3d:
    subsampled_data[vbl]=ds[vbl].interp(survey_indices)
# Interpolate U and V from i_g, j_g to i, j, then interpolate:
# Get u, v
grid = Grid(ds, coords={'X':{'center': 'i', 'left': 'i_g'}, 
                        'Y':{'center': 'j', 'left': 'j_g'},
                        'Z':{'center': 'k'}})
U_c = grid.interp(ds.U, 'X', boundary='extend')
V_c = grid.interp(ds.V, 'Y', boundary='extend')
subsampled_data['U'] = U_c.interp(survey_indices)
subsampled_data['V'] = V_c.interp(survey_indices)    
# # subsampled_data['U_test']=ds['U'].interp(survey_indices)
# # ds
# survey_indices
# grid = Grid(ds, coords={'X':{'center': 'i', 'left': 'i_g'}, 
#                         'Y':{'center': 'j', 'left': 'j_g'},
#                         'Z':{'center': 'k'}})
# U_c = grid.interp(ds.U, 'X', boundary='extend')
# V_c = grid.interp(ds.V, 'Y', boundary='extend')
subsampled_data['U_test']=ds['U'].interp(survey_indices)


### TEST! is it faster to interpolate mooring data to a grid rather than as "points"?


In [ ]:
sampling_parameters

In [ ]:
# interpolate to xi,yi,ti,zi
xi = sampling_parameters['xmooring']
yi = sampling_parameters['ymooring']
ti = ds['time']
zi = sampling_parameters['zmooring_TS']

# ---- copied ----
# Convert lon, lat and z to index i, j and k with f_x, f_y and f_z
# XC, YC and Z are the same at all times, so select a single time
X = ds.XC.isel(time=0) 
Y = ds.YC.isel(time=0)
i = ds.i
j = ds.j
z = ds.Z.isel(time=0)
k = ds.k
f_x = interpolate.interp1d(X[0,:].values, i)
f_y = interpolate.interp1d(Y[:,0].values, j)
f_z = interpolate.interp1d(z, k, bounds_error=False)
# ---- copied ----
    
# survey_indices_mooring= xr.Dataset(
#         dict(
#             i = xr.DataArray(f_x(xmooring), dims='points'),
#             j = xr.DataArray(f_y(ymooring), dims='points'),
#             k = xr.DataArray(f_z(survey_track.dep), dims='points'),
#             time = xr.DataArray(survey_track.time, dims='points'),
#         )
#     )

In [ ]:
# -- initialize the dataset:
# mgridded = xr.Dataset(
#     coords = dict(depth=(["depth"],zgridded),
#               time=(["time"],times))
# # )
# vbls3d = ['Theta','Salt','vorticity','steric_height', 'U', 'V']
# vbls3d = ['Theta','Salt','vorticity','steric_height', 'U_c', 'V_c']
# for vbl in vbls3d:
#     print('interpolating ' + vbl)
#     mgridded[vbl]=ds[vbl].interp(i=f_x(xi), j=f_y(yi), k=f_z(zi), time=ti).compute()
   
ds

In [ ]:
plt.pcolormesh(np.transpose(dum.data))
plt.show()

In [ ]:
sgridded['Theta'].plot()

In [ ]:

# try regridding U/V earlier (when deriving)

In [ ]:
# sgridded.oceTAUX.plot()
plt.quiver(sgridded.time.data,0,sgridded.oceTAUX.data, sgridded.oceTAUY.data)

In [ ]:
# plot to compare the two methods 
# ( not quite working )
tpl = ds['time']/24,sss[:,0,0,0]
ttt=ds['time']
# ipl = ( subsampled_data.dep.values == subsampled_data.dep.values.min() )
# plt.plot(subsampled_data.time(ipl),subsampled_data.Salt.sel(ipl).data,'-')
# plt.plot(subsampled_data.time,subsampled_data.Salt.data,'-')
plt.plot(tgr,np.transpose(dum),'-')
plt.plot(ttt[:],sss[:,0,0,0],'k--')
# plt.legend('survey_interp method','direct interpolation')

### Visualize steric height

In [ ]:
%matplotlib inline
sh_anom = sh_true.values - sh_true.values.mean()
plt.figure(figsize=(7,5))
sho = plt.scatter(survey_track.lon, survey_track.lat, c=sh_anom)
plt.title('Steric height anomaly')
plt.colorbar(sho).set_label('m')
plt.show()

### Comparison of true vs sampled steric height
Plot comparing the "true" steric height along the track and the subsampled steric height, which is computed by integrating the specific volume anomaly for each subsampled profile from its deepest sampling depth

In [ ]:
%matplotlib inline

# truth:
plt.plot(sh_true.time, sh_true.values - sh_true.values.mean())
# get index of the deepest sampling depths
i = ( subsampled_data.dep.values == subsampled_data.dep.values.min() )
plt.plot(subsampled_data.time.values[i], subsampled_data.steric_height.values[i] - subsampled_data.steric_height.values[i].mean(),'.-')
plt.title('Steric height anomaly along the survey track')
plt.legend(['truth','subsampled data'])
plt.xlabel('time, days')
plt.ylabel('steric height anom., m')

In [ ]:
# NEW CODE  BELOW

# interpolation:
subsampled_data = xr.Dataset()  

# loop & interpolate through 3d variables:
vbls3d = ['Theta','Salt','vorticity']
# vbls3d = ['Theta']
for vbl in vbls3d:
    subsampled_data[vbl]=ds[vbl].interp(survey_indices)
# # Interpolate U and V from i_g, j_g to i, j, then interpolate:
# U_c = grid.interp(ds.U, 'X', boundary='extend')
# V_c = grid.interp(ds.V, 'Y', boundary='extend')
# subsampled_data['U'] = U_c.interp(survey_indices)
# subsampled_data['V'] = V_c.interp(survey_indices)

subsampled_data['lon']=survey_track.lon
subsampled_data['lat']=survey_track.lat
subsampled_data['dep']=survey_track.dep
subsampled_data['time']=survey_track.time 

# loop & interpolate through 2d variables:
vbls2d = ['Eta', 'KPPhbl', 'PhiBot', 'oceFWflx', 'oceQnet', 'oceQsw', 'oceSflux', 'oceTAUX', 'oceTAUY']
vbls2d = ['Eta', 'Depth']
# create 2-d survey track by removing the depth dimension
survey_indices_2d =  survey_indices.drop_vars('k')
for vbl in vbls2d:
    subsampled_data[vbl]=ds[vbl].interp(survey_indices_2d)
# survey_indices_2d.i.plot()
# ds['KPPhbl'].interp(survey_indices_2d).plot()

# interp
# this returns a value at every timestep (points) - very high resolution
# - could subsample to the model time grid 


# plot
# plt.plot(sh_true.time, sh_true.values - sh_true.values.mean())
# plt.plot(subsampled_data.time, ssh - ssh.mean())


In [ ]:
subsampled_data = xr.Dataset() 

# loop & interpolate through 3d variables:
vbls3d = ['Theta','Salt','vorticity','steric_height']
for vbl in vbls3d:
    subsampled_data[vbl]=ds[vbl].interp(survey_indices)
# Interpolate U and V from i_g, j_g to i, j, then interpolate:
# Get u, v
grid = Grid(ds, coords={'X':{'center': 'i', 'left': 'i_g'}, 
                        'Y':{'center': 'j', 'left': 'j_g'},
                        'Z':{'center': 'k'}})
U_c = grid.interp(ds.U, 'X', boundary='extend')
V_c = grid.interp(ds.V, 'Y', boundary='extend')
subsampled_data['U'] = U_c.interp(survey_indices)
subsampled_data['V'] = V_c.interp(survey_indices)


# add lat/lon/time to dataset
subsampled_data['lon']=survey_track.lon
subsampled_data['lat']=survey_track.lat
subsampled_data['dep']=survey_track.dep
subsampled_data['time']=survey_track.time  

# loop & interpolate through 2d variables:
vbls2d = ['Eta', 'KPPhbl', 'PhiBot', 'oceFWflx', 'oceQnet', 'oceQsw', 'oceSflux']
# create 2-d survey track by removing the depth dimension
survey_indices_2d =  survey_indices.drop_vars('k')
for vbl in vbls2d:
    subsampled_data[vbl]=ds[vbl].interp(survey_indices_2d)   
# taux & tauy must be treated like U and V
oceTAUX_c = grid.interp(ds.oceTAUX, 'X', boundary='extend')
oceTAUY_c = grid.interp(ds.oceTAUY, 'Y', boundary='extend')
subsampled_data['oceTAUX'] = oceTAUX_c.interp(survey_indices_2d)
subsampled_data['oceTAUY'] = oceTAUY_c.interp(survey_indices_2d)


    

In [ ]:
ds.U

In [ ]:
plt.plot(sh_true.time, sh_true.values - sh_true.values.mean())
plt.plot(subsampled_data.time, ssh - ssh.mean())


In [ ]:
# mooring test

model_xav = ds.XC.cccc.mean(dim='i').values
model_yav = ds.YC.isel(time=0, i=0).mean(dim='j').values

xmooring = model_xav # default lat/lon is the center of the domain
ymooring = model_yav
zmooring_TS = [-1, -10, -50, -100] # depth of T/S instruments
zmooring_UV = [-1, -10, -50, -100, -200, -300, -400, -500] # depth of U/V instruments


ts = ds.time
n_samples = ts.size
n_depths_TS = np.size(zmooring_TS)
n_depths_UV = np.size(zmooring_UV)
# depth sampling - different for TS and UV
zs_TS = np.transpose(np.tile(zmooring_TS,(n_samples,1)))
zs_UV = np.transpose(np.tile(zmooring_UV,(n_samples,1)))
xs_TS = xmooring * np.ones([n_depths_TS, n_samples])
ys_TS = ymooring * np.ones([n_depths_TS, n_samples])
xs_UV = xmooring * np.ones([n_depths_TS, n_samples])
ys_UV = ymooring * np.ones([n_depths_TS, n_samples])

survey_track = xr.Dataset(
    dict(
        lon = xr.DataArray(xs_TS,dims='points'),
        lat = xr.DataArray(ys_TS,dims='points'),
        dep = xr.DataArray(zs_TS,dims='points'),
        time = xr.DataArray(ts_TS,dims='points')
    )
)



In [ ]:
# np.broadcast_to([-1, -10, -50, -100],(2,4))
np.broadcast_to([-1, -10, -50, -100],(2,4))

In [ ]:
# np.broadcast_to(zmooring_TS,(2,4))
# np.tile(zmooring_TS.transpose,(2,1))
# np.tile(np.transpose(zmooring_TS),(1,n_samples)).shape
# xs_TS.shape
# [-1, -10, -50, -100]
# zmooring_TS

# np.transpose(np.array(zmooring_TS)).shape 
np.transpose(np.tile(zmooring_TS,(n_samples,1))).shape
xs_TS.shape

In [ ]:
subsampled_data

In [ ]:
plt.scatter(subsampled_data.time,subsampled_data.dep,c=subsampled_data.Theta)
plt.plot(subsampled_data.time,-subsampled_data.Depth,c='r')

In [ ]:
sh_true.plot()

In [ ]:

# we interpolated everything to the "points" - one datapoint per sample
# but we may also want a cleaner (gridded) output product that has been reshaped into 
# profiles (i.e., a X x Z x T array)
# (This might not make sense for glider data, since profiles are likely > one model gridpoint)

# separate these into "subsampled_points" and "subsampled_profiles"?

# a couple ways to determine the profiles:
# - one per down / one per up, with time as either the start/mean/end
# - one per down-up, with the time as the deepest point
# - maybe others? would be useful to get feedback on this...

# use "where" to determine the indices of the start/end (shallowest/deepest) of each profile:

# subsampled_data.steric_height.where(subsampled_data.dep == subsampled_data.dep.min(), drop=True)
# subsampled_deepest = subsampled_data.where(subsampled_data.dep == subsampled_data.dep.min(), drop=True)

# plt.plot(subsampled_data.dep.values[ideep])

# this is the DEEPEST point only. 
subsampled_data = subsampled_data
dum = subsampled_data.where(subsampled_data.dep == subsampled_data.dep.min(), drop=True)
shall = subsampled_data.where(subsampled_data.dep == subsampled_data.dep.max(), drop=True)
%matplotlib inline
plt.figure(figsize=(7,5))
plt.plot(dum.lon,dum.lat,'.')
plt.plot(shall.lon,shall.lat,'.')
plt.show()


In [ ]:
import time
# index?
ishallow = ( subsampled_data.dep.values == subsampled_data.dep.values.max() )
ideep = ( subsampled_data.dep.values == subsampled_data.dep.values.min() )
# boolean to index
ishallow = [ishallow for ishallow, x in enumerate(ishallow) if x]
ideep = [ideep for ideep, x in enumerate(ideep) if x]

t_profiles = subsampled_data['time'].isel(points=ishallow)
t_profiles.plot()
z = np.unique(subsampled_data['dep'])

# # # initialize the dataset:
# subsampled_profile = xr.Dataset(
#     coords={
#         "time": t_profiles,
#         "depth": z
#     },
#     "Salt": (("time", "depth"), []),
# )


# loop through each profile:

pr = []
for n in np.arange(np.size(ishallow)):
    i = np.arange(ishallow[n],ideep[n])
    # append
    pr = np.append(pr,subsampled_data['Salt'].isel(points=i))
    
#     # loop & interpolate through 3d variables:
#     for vbl in vbls3d:
# #         subsampled_profile[vbl] = subsampled_data[vbl].isel(points=i)
#         dum = subsampled_data[vbl].isel(points=i)

pr   
        

    

In [ ]:
n

In [ ]:
np.shape(pr)

In [ ]:

# # n=0
# # i = np.arange(ishallow[n],ideep[n])
# # start = time.time()
# # dum = subsampled_data.Theta.isel(points=i)
# # end = time.time()
# # print("The time of execution of above program is :", end-start)

In [ ]:
subsampled_data.groupby("dep").mean().scatter()


In [ ]:
ss = xr.Dataset(
    {
        "Salt": (("time", "depth"), [subsampled_data.Salt.isel(points=i)]),
    },
    coords={
        "time": [1],
        "depth": z,
    },
    
)
ss
# "precipitation": (("lat", "lon"), np.random.rand(4).reshape(2, 2)),

In [ ]:
len(z)

In [ ]:
subsampled_data

In [ ]:
# {**sampling_details}


def test_code(sampling_details):
    for key, value in sampling_details.items():
#         print(key , '=' , value , '')
#         print(type(key))
        if isinstance(value,str):
            exec(key + '="' + value + '"',None, globals())
#             print(key)
            
        if isinstance(value,list):
            exec(key + '=' + str(value) + '',locals(), globals())
#             print(key)
            
    print(zrange+1000)
    return zrange
    
ddd = test_code(sampling_details)
# test_code(sampling_details)
ddd
# for key in sampling_details:
#     exec(key + '=' + '"' + sampling_details[key] + '"')

# isinstance(sampling_details['zrange'],list)
# isinstance(sampling_details['SAMPLING_STRATEGY'],list)
# isinstance?

# type(sampling_details['zrange'])

In [ ]:

#             # 3) merge transformed data with ds
#             print('merging with ds')
#             print(ds)
#             print(oceTAUX_r)
#             ds = ds.merge(U_r.to_dataset(name='U_r')).merge(V_r.to_dataset(name='V_r'))
#             ds = ds.merge(oceTAUX_r.to_dataset(name='oceTAUX_r')).merge(oceTAUY_r.to_dataset(name='oceTAUY_r'))

#             # 4) rename transformed data to original names (renaming to 'temp' first)
#             # !! caution, skipping this step will cause great confusion later !!
#             ds = ds.rename_vars({'U':'Utemp', 'V':'Vtemp'}).rename_vars({'U_r':'U', 'V_r':'V'})
#             ds = ds.rename_vars({'oceTAUX':'TAUXtemp', 'oceTAUY':'TAUYtemp'}).rename_vars({'oceTAUX_r':'oceTAUX', 'oceTAUY_r':'oceTAUY'})
           

#             # 6) get rid of all the original U/V/TAUX/TAUY variables (which have been renamed to *temp)
#             ds = ds.drop_vars({'Utemp','Vtemp','TAUXtemp','TAUYtemp'})
            


In [ ]:
# next two cells: explore looping through files & using concat instead of opening w open_mfdataset

In [ ]:
%%time
# or - try loading in a loop and using concat:
ds2 = xr.open_dataset(target_files[0], drop_variables={'U', 'V', 'oceTAUX', 'oceTAUY'},
                     chunks={'i':xchunk, 'j':ychunk, 'time':tchunk})
for fn in target_files[1:-1]:
    thisds = xr.open_dataset(fn, drop_variables={'U', 'V', 'oceTAUX', 'oceTAUY'},
                            chunks={'i':xchunk, 'j':ychunk, 'time':tchunk})
    ds2 = xr.concat([ds2, thisds], dim='time')




In [ ]:
%%time

#ok, so what if we load/concat target_files and derived_files - which crashes if we use open_mfdataset

# load the corresponding derived fields (includes steric height, vorticity, and transformed vector variables for current and wind stress)
derivedir = datadir + 'derived/'
derived_files = [f'{derivedir}LLC4320_pre-SWOT_{RegionName}_derived-fields_{date_list[n].strftime("%Y%m%d")}.nc' for n in range(ndays)] # list target files

ds_all = xr.open_dataset(target_files[0], drop_variables={'U', 'V', 'oceTAUX', 'oceTAUY'},
                     chunks={'i':xchunk, 'j':ychunk, 'time':tchunk})
dsd = xr.open_dataset(derived_files[0], chunks={'i':xchunk, 'j':ychunk, 'time':tchunk})
ds_all = ds_all.merge(dsd)
n = 0
for fn in target_files[1:-1]:
    # original data:
    thisds = xr.open_dataset(fn, drop_variables={'U', 'V', 'oceTAUX', 'oceTAUY'},
                            chunks={'i':xchunk, 'j':ychunk, 'time':tchunk})
    # derived file. 
    n+=1
    thisdsd = xr.open_dataset(derived_files[n], chunks={'i':xchunk, 'j':ychunk, 'time':tchunk})
    # merge the two:
    thisds = thisds.merge(thisdsd)
    # and concat with all files
    ds_all = xr.concat([ds_all, thisds], dim='time')
# rename the transformed vector variables to their original names
ds_all = ds_all.rename_vars({'U_transformed':'U', 'V_transformed':'V',
                             'oceTAUX_transformed':'oceTAUX', 'oceTAUY_transformed':'oceTAUY'})
# Convert lon, lat and z to index i, j and k with f_x, f_y and f_z
# XC, YC and Z are the same at all times, so select a single time
X = ds_all.XC.isel(time=0) 
Y = ds_all.YC.isel(time=0)


### Test using xoak:
This seems to give approx equivalent results as my method, but I can't figure out how to optimize the parameters so that it doesn't crash the kernel ... so I will put this aside for now & stick with the orginal method.

In [ ]:
# have to create a new dataset to do use xoak, because the coords have to have the same dims as the variables


# time has to be a float
ds['timef'] = ds.time.copy()  # in datetime64 format
# converting date time into second timestamp 
tt = ds.timef
tt = (tt - tt[0]) / np.timedelta64(1, 'h')
ds = ds.assign_coords(timef=tt)

# create coords that all have the same shape as the data:
# for 3d variables (T x Z x Y x X)
X3d = ds.XC.broadcast_like(ds.Z)
Y3d = ds.YC.broadcast_like(ds.Z)
T3d = ds.timef.broadcast_like(X3d)
Z3d = ds.Z.broadcast_like(X3d)
# for 2d variables (T x Z x Y)
X2d = ds.XC # already has the right dimensions
Y2d = ds.YC # already has the right dimensions
T2d = ds.timef.broadcast_like(X2d)

# # => to use dask, chunk the data.
# chunk_shape = (ds.Theta.chunks[0][0], ds.Theta.chunks[1][0], ds.Theta.chunks[2][0], ds.Theta.chunks[3][0])

# create a new dataset with the variables to interpolate and the 4-d coordinates:
dsoak_3d = xr.Dataset(
    coords={'X': (('time', 'k', 'j', 'i'), X3d.values), 'Y': (('time', 'k', 'j', 'i'), Y3d.values), 
            'T': (('time', 'k', 'j', 'i'), T3d.values), 'Z': (('time', 'k', 'j', 'i'), Z3d.values)},
    data_vars={'Theta': (('time', 'k', 'j', 'i'), ds.Theta.data)},
)
dsoak_3d


In [ ]:
# rechunk (LEADS TO AN ERROR IN xoak.set_index)
# dsoak_3d = dsoak_3d.chunk({"time": tchunk, "j":ychunk, "i":xchunk})
# dsoak_3d

In [ ]:
# same for 2d:
# create a new dataset with the variables to interpolate and the 4-d coordinates:
dsoak_2d = xr.Dataset(
    coords={'X': (('time', 'j', 'i'), X2d.values), 'Y': (('time', 'j', 'i'), Y2d.values), 
            'T': (('time', 'j', 'i'), T2d.values)},
    data_vars={'Eta': (('time', 'j', 'i'), ds.Eta.data)},
)
dsoak_2d

In [ ]:
%%time
# xoak: set index
# options are:
#     scipy_kdtree           --  fails with an error
#     sklearn_kdtree         --  kernel crashes 
#     sklearn_balltree       --  this is the only one that works, but nearest neighbor - coarse and not accurate?
#     sklearn_geo_balltree   --  doesn't work except with lat/lon
#     s2point                --  seems to give a single point at the start/end!
#     brute_force            --  kernel crashes

dsoak_3d.xoak.set_index(['X', 'Y', 'T', 'Z'], 'sklearn_balltree') 
# should have one object per chunk, but there's only one object so I guess it only created one chunk
# however, if I rechunk earlier it gives an error at this step.
dsoak_3d.xoak.index

In [ ]:
# convert time in survey_track hours (since start of the time in dsoak_3d)

# time has to be a float
survey_track['timef'] = survey_track.time.copy()  # in datetime64 format
# converting date time into second timestamp 
ttrack = survey_track.timef
ttrack = (ttrack - ds.time.data[0]) / np.timedelta64(1, 'h') # ref time time[0] for the dataset
survey_track = survey_track.assign_coords(timef=ttrack)

In [ ]:
# can chunk survey_track
# - this seems to improve performance but also leads to a kernel restart, so.... maybe I have to play with the params
# survey_track
# survey_track = survey_track.chunk({"points": 500})

In [ ]:
from dask.diagnostics import ProgressBar

with ProgressBar(), dask.config.set(scheduler='processes'):
    subsampled_xoak_3d = dsoak_3d.xoak.sel(
        X=survey_track.lon,
        Y=survey_track.lat,
        T=survey_track.timef,
        Z=survey_track.dep
    )

subsampled_xoak_3d


In [ ]:
# create dep, lat, lon, time variables to be consistent with subsampled_data:
subsampled_xoak_3d['dep'] = subsampled_xoak_3d.Z
subsampled_xoak_3d['lat'] = subsampled_xoak_3d.Y
subsampled_xoak_3d['lon'] = subsampled_xoak_3d.X
subsampled_xoak_3d['time'] = subsampled_xoak_3d.T
subsampled_xoak_3d

In [ ]:
subsampled_data['timef'] = subsampled_data.time.copy()  # in datetime64 format
# converting date time into second timestamp 
ttt = subsampled_data.time
ttt = (ttt - ds.time.data[0]) / np.timedelta64(1, 'h') # ref time time[0] for the dataset
subsampled_data = subsampled_data.assign_coords(timef=ttt)

plt.plot(subsampled_data.timef, subsampled_data.dep,'.')
plt.plot(subsampled_xoak_3d.time, subsampled_xoak_3d.dep,'.')
plt.show()

In [ ]:
plt.plot(subsampled_data.Theta,'.')
plt.plot(subsampled_xoak_3d.Theta,'.')
plt.show()

In [ ]:
# regrid
# since we've done a nearest neighbor interp, the data are oversampled
# ... can we just reshape?
# yes, but the grid can't be linear
# *and* it can't be unique

# => for our example, just figure out length of zgridded by plotting subsampled_xoak_3d['dep']

nz = int(1998/2)
zgridded = subsampled_xoak_3d['dep'].isel(points=slice(0,nz))

# cut length to an integer number of nz:
subsampled_xoak_3d = subsampled_xoak_3d.isel(points=slice(0,int(np.floor(subsampled_xoak_3d.points.size/nz)*nz)))
times = subsampled_xoak_3d['time'].isel(points=np.arange(0,len(subsampled_xoak_3d.time),nz))
nt = len(times)


In [ ]:

# -- initialize the gridded dataset:
sgridded_xoak_3d = xr.Dataset(
    coords = dict(depth=(["depth"],zgridded.data),
              time=(["time"],times.data))
)
# -- 3-d fields: loop & reshape 3-d data from profiles to a 2-d (depth-time) grid:
# first, extract each variable, then reshape to a grid
vbls3d = ['Theta']

SAMPLING_STRATEGY = sampling_parameters['SAMPLING_STRATEGY']

for vbl in vbls3d:
    print(vbl)
    this_var = subsampled_xoak_3d[vbl].data.compute().copy() 
    # reshape to nz,nt
    this_var_reshape = np.reshape(this_var,(nz,nt), order='F') # fortran order is important!
    # for platforms with up & down profiles (uCTD and glider),
    # every second column is upside-down (upcast data)
    # starting with the first column, flip the data upside down so that upcasts go from top to bottom
    if SAMPLING_STRATEGY != 'sim_mooring':
        this_var_fix = this_var_reshape.copy()
        #this_var_fix[:,0::2] = this_var_fix[-1::-1,0::2] 
        this_var_fix[:,1::2] = this_var_fix[-1::-1,1::2]  # Starting with SECOND column
        sgridded_xoak_3d[vbl] = (("depth","time"), this_var_fix)
    elif SAMPLING_STRATEGY == 'sim_mooring':
        sgridded_xoak_3d[vbl] = (("depth","time"), this_var_reshape)


In [ ]:
# plot
sgridded_xoak_3d.Theta.plot()

In [ ]:
# original method
sgridded.Theta.plot()


# TESTING!

concat() has a number of options which provide deeper control over which variables are concatenated and how it handles conflicting variables between datasets. With the default parameters, xarray will load some coordinate variables into memory to compare them between datasets. This may be prohibitively expensive if you are manipulating your dataset lazily using Parallel computing with Dask.


In [ ]:
# %%time
# # instead of loading all with open_mfdataset, load in a loop?

# #ok, so what if we load/concat target_files and derived_files - which crashes if we use open_mfdataset

# # load the corresponding derived fields (includes steric height, vorticity, and transformed vector variables for current and wind stress)
# derivedir = datadir + 'derived/'
# derived_files = [f'{derivedir}LLC4320_pre-SWOT_{RegionName}_derived-fields_{date_list[n].strftime("%Y%m%d")}.nc' for n in range(ndays)] # list target files

# ds_all = xr.open_dataset(target_files[0], drop_variables={'U', 'V', 'oceTAUX', 'oceTAUY'},
#                      chunks={'i':xchunk, 'j':ychunk, 'time':tchunk})
# # dsd = xr.open_dataset(derived_files[0], chunks={'i':xchunk, 'j':ychunk, 'time':tchunk})
# # ds_all = ds_all.merge(dsd)
# n = 0
# for fn in target_files[1:-1]:
#     # original data:
#     thisds = xr.open_dataset(fn, drop_variables={'U', 'V', 'oceTAUX', 'oceTAUY'},
#                             chunks={'i':xchunk, 'j':ychunk, 'time':tchunk})
# #     # derived file. 
# #     n+=1
# #     thisdsd = xr.open_dataset(derived_files[n], chunks={'i':xchunk, 'j':ychunk, 'time':tchunk})
# #     # merge the two:
# #     thisds = thisds.merge(thisdsd)
#     # and concat with all files
        
#     ds_all = xr.concat([ds_all, thisds], dim='time', 
#                        data_vars={'XC', 'YC', 'Z', 'Eta', 'KPPhbl', 'PhiBot', 'oceFWflx', 
#                                   'oceQnet', 'oceQsw', 'oceSflux', 'Theta', 'Salt', 'W'},
#                        coords={'time'},
#                        compat='override', join='override', combine_attrs='override')

# #                        coords={'j_g', 'i', 'i_g', 'j', 'k', 'nb', 'time'}, 
        
# # rename the transformed vector variables to their original names
# # ds_all = ds_all.rename_vars({'U_transformed':'U', 'V_transformed':'V',
# #                              'oceTAUX_transformed':'oceTAUX', 'oceTAUY_transformed':'oceTAUY'})
# # Convert lon, lat and z to index i, j and k with f_x, f_y and f_z
# # XC, YC and Z are the same at all times, so select a single time
# X = ds_all.XC.isel(time=0) 
# Y = ds_all.YC.isel(time=0)

# ds = ds_all.copy()

In [ ]:
# TESTING !!!
vbls3d = ['Theta','Salt','vorticity','steric_height', 'U', 'V']
vbls2d = ['steric_height_true', 'Eta', 'KPPhbl', 'PhiBot', 'oceTAUX', 'oceTAUY', 'oceFWflx', 'oceQnet', 'oceQsw', 'oceSflux']
        
vbls3d = ['Theta']
  
# -- for glider/uctd, take the shallowest & deepest profiles (every second value, since top/bottom get sampled twice for each profile)
time_deepest = subsampled_data.time.where(subsampled_data.dep == subsampled_data.dep.min(), drop=True).values[0:-1:2]
time_shallowest = subsampled_data.time.where(subsampled_data.dep == subsampled_data.dep.max(), drop=True).values[0:-1:2]
times = np.sort(np.concatenate((time_shallowest, time_deepest)))
# this results in a time grid that may not be uniformly spaced, but is correct
# - for a uniform grid, use the mean time spacing - may not be perfectly accurate, but is evenly spaced
dt = np.mean(np.diff(time_shallowest))/2 # average spacing of profiles (half of one up/down, so divide by two)
times_uniform = np.arange(survey_track.n_profiles.values*2) * dt

# nt is the number of profiles (times):
nt = len(times)  
# xgr is the vertical grid; nz is the number of depths for each profile
# depths are negative, so sort in reverse order using flip
zgridded = np.flip(np.unique(subsampled_data.dep.data))
nz = int(len(zgridded))



In [ ]:
%%time
# -- initialize the dataset:
sgridded = xr.Dataset(
    coords = dict(depth=(["depth"],zgridded),
              time=(["time"],times))
)
# -- 3-d fields: loop & reshape 3-d data from profiles to a 2-d (depth-time) grid:
# first, extract each variable, then reshape to a grid
vbls3d = ['Theta']
for vbl in vbls3d:
    print(vbl)
    %time this_var = subsampled_data[vbl].data.compute().copy() 
    # reshape to nz,nt
    %time this_var_reshape = np.reshape(this_var,(nz,nt), order='F') # fortran order is important!
    # for platforms with up & down profiles (uCTD and glider),
    # every second column is upside-down (upcast data)
    # starting with the first column, flip the data upside down so that upcasts go from top to bottom
    if SAMPLING_STRATEGY != 'sim_mooring':
        this_var_fix = this_var_reshape.copy()
        #this_var_fix[:,0::2] = this_var_fix[-1::-1,0::2] 
        this_var_fix[:,1::2] = this_var_fix[-1::-1,1::2]  # Starting with SECOND column
        sgridded[vbl] = (("depth","time"), this_var_fix)
    elif SAMPLING_STRATEGY == 'sim_mooring':
        sgridded[vbl] = (("depth","time"), this_var_reshape)


# # for sampled steric height, we want the value integrated from the deepest sampling depth:
# sgridded['steric_height'] = (("time"), sgridded['steric_height'].isel(depth=nz-1).data)
# # rename to "steric_height_sampled" for clarity
# sgridded.rename_vars({'steric_height':'steric_height_sampled'})



# #  -- 2-d fields: loop & reshape 2-d data to the same time grid 
# for vbl in vbls2d:
#     this_var = subsampled_data[vbl].data.compute().copy() 
#     # subsample to nt
#     this_var_sub = this_var[0:-1:nz]
#     sgridded[vbl] = (("time"), this_var_sub)

In [ ]:
# this is the slow step: subsampled_data[vbl].data.compute().copy()
# - can we do this differently?
%time this_var = subsampled_data[vbl].data
# reshape to nz,nt
# %time this_var_reshape = np.reshape(this_var,(nz,nt), order='F') # fortran order is important!
%time this_var_reshape = np.reshape(this_var,(nz,nt))

In [ ]:
%time dum = this_var_reshape.compute()

# end testing!